In [1]:
# control execution time of functions
import threading

TIMER = 120
PERMUT = 12

class TimeoutError(Exception):
    pass

class InterruptableThread(threading.Thread):
    def __init__(self, func, *args, **kwargs):
        threading.Thread.__init__(self)
        self._func = func
        self._args = args
        self._kwargs = kwargs
        self._result = None

    def run(self):
        self._result = self._func(*self._args, **self._kwargs)

    @property
    def result(self):
        return self._result


class timeout(object):
    def __init__(self, sec):
        self._sec = sec

    def __call__(self, f):
        def wrapped_f(*args, **kwargs):
            it = InterruptableThread(f, *args, **kwargs)
            it.start()
            it.join(self._sec)
            if not it.is_alive():
                return it.result
            raise TimeoutError('execution expired')
        return wrapped_f

In [2]:
def make_generator(parameters):
    """Helper function for st_grid_search. Returns a dictionary of all possible parameter combinations."""
    if not parameters:
        yield dict()
    else:
        key_to_iterate = list(parameters.keys())[0]
        next_round_parameters = {p : parameters[p]
                    for p in parameters if p != key_to_iterate}
        for val in parameters[key_to_iterate]:
            for pars in make_generator(next_round_parameters):
                temp_res = pars
                temp_res[key_to_iterate] = val
                yield temp_res


@timeout(TIMER*PERMUT)
def st_grid_search(estimator, split, X, param_dict, metric, n_clusters, y=None, frame_size=None, frame_overlap=None):
    """
    Grid Search of hyperparameters for spatial-temporal clustering algorithms
    
    Parameters
    ----------
    estimator: class
        ST clustering algorithm
    split: boolean
        Flag to indicate whether whole X should be loaded in RAM or processed in smaller chunks.
    X: numpy array
        Data on which grid search is performed
    param_dict: dict
        Dictionary with parameters to be optimized as keys and value range of grid search as value.
    metric: str
        The metric to evaluate the clustering quality
    y: numpy array
        Optional. Some metrics compare predictions with ground truth. Then, labels need to be provided.
    frame_size: int
        Optional. If split is True, indicate how large the chunks should be.
    
    Returns
    -------
    param_opt
        Optimal hyperparameter combination
    """
    param_opt = None
    s_max = 0
    
    t,n, m_plus = X.shape
    st_X = X.reshape(t*n, m_plus)
    
    for param in make_generator(param_dict):
        clust = estimator(**param)
        
        if not split:
            clust.st_fit(st_X)
        else:
            clust.st_fit_frame_split(st_X, frame_size, frame_overlap)
            
        if param_opt is None: 
            param_opt = param
        
        # different performance evaluation metrics
        
        if metric=='long_term_ami':
            weights = clust.labels.reshape((t,n)).T
            score = find_optimal_threshold(weights, n_clusters, y)
#             score = adjusted_mutual_info_score(y,clust.labels)
        # store parameter combination if it outperforms given the metric
    
        elif metric=='full_ami':
            score = adjusted_mutual_info_score(np.tile(y,t), clust.labels)
            
        elif metric =='average_ami':
            weights = clust.labels.reshape((t,n)).T
            lt_score = find_optimal_threshold(weights, n_clusters, y)
            full_score = adjusted_mutual_info_score(np.tile(y,t), clust.labels)
            
            score = (lt_score + full_score)/2
            
        if score > s_max:
            s_max = score
            param_opt = param
    return param_opt



In [5]:
class Test(object):       
    # use this function with st clusterers
    @timeout(TIMER) # set seconds for timeout
    def frame_split_cluster(self, algorithm, data, frame_size, frame_overlap):
        t,n, m_plus = data.shape
        st_data = data.reshape(t*n, m_plus)
        start_time = time.time()
        algorithm.st_fit_frame_split(st_data, frame_size, frame_overlap)
        runtime = time.time() - start_time
        weights = algorithm.labels.reshape((t,n)).T
        long_term_ami, full_ami= find_optimal_threshold(weights, k, true_labels)
#         full_ami = adjusted_mutual_info_score(np.tile(true_labels, t), algorithm.labels)
#         ami = adjusted_mutual_info_score(labels, algorithm.labels)
        return long_term_ami, full_ami, runtime

    def frame_split_cluster_fill_out(self, algorithm, data, frame_size, frame_overlap, t_full):
        t,n, m_plus = data.shape
        st_data = data.reshape(t*n, m_plus)
        start_time = time.time()
        algorithm.st_fit_frame_split(st_data, frame_size, frame_overlap)
        runtime = time.time() - start_time
        weights = algorithm.labels.reshape((t,n)).T
        assignments = fill_out_predictions(weights, t_full)
        long_term_ami, full_ami = find_optimal_threshold(assignments, k, true_labels)
#         full_ami = adjusted_mutual_info_score(np.tile(true_labels, t), algorithm.labels)
#         ami = adjusted_mutual_info_score(labels, algorithm.labels)
        return long_term_ami, full_ami, runtime
        
    # use this with trajectory clustering
    @timeout(TIMER)
    def traj_cluster(self,algorithm, data):
        start_time = time.time()
        algorithm.st_fit(data)
        runtime = time.time() - start_time
        ami = adjusted_mutual_info_score(algorithm.true_labels, algorithm.labels)
        return ami, runtime
        
    # use this with dbscan2
    @timeout(TIMER)
    def cluster(self, algorithm, data):
        start_time = time.time()
        algorithm.st_fit(data)
        runtime = time.time() - start_time
        ami = adjusted_mutual_info_score(labels, algorithm.labels)
        return ami, runtime

In [4]:
import numpy as np
import pandas as pd
import os
import time
import logging
import json
from scipy.spatial.distance import pdist, squareform
from sklearn.metrics import silhouette_score, adjusted_mutual_info_score
from st_clustering import ST_DBSCAN, ST_Agglomerative, ST_KMeans, ST_SpectralClustering, ST_AffinityPropagation, ST_BIRCH, ST_HDBSCAN, ST_OPTICS
from TKM_long_term_clusters import find_optimal_threshold



In [7]:
FRAME_SIZE = 100
FRAME_OVERLAP = 10

sizes = [800, 3000, 6000, 10000, 15000, 20000, 25000, 35000]#, 50000, 80000, 100000, 130000, 150000, 180000, 200000]
#[800, 3000, 6000, 10000, 15000, 20000, 25000, 35000,
methods = ['dbscan', 'hdbscan']#, 'kmeans', 'birch']#['dbscan','agglomerative', 'kmeans', 'hdbscan', 'birch']

eps1 = 0.50
alpha = 1

# subset_option = False

for ind in range(len(sizes) -1):
    min_size = sizes[ind]
    max_size = sizes[ind+1]

    for method in methods: 

        metric = 'static'#'average_ami'

        f = open(method + '_' + metric + "_log_%d_%d_1.txt" %(min_size, max_size), "w")
        lt_ami =[]
        full_ami_list = []
        runtime_list=[]

        for filename in os.listdir("../../full_data/"):

            # read data
            if filename.endswith(".csv"):


                df = pd.read_csv('../../full_data/'+filename)

                dataset_size = df['id'].nunique()*df['frame'].nunique()

                if dataset_size in range(min_size, max_size):

                    df['x'] = (df['x'] - df['x'].min()) / (df['x'].max() - df['x'].min())
                    df['y'] = (df['y'] - df['y'].min()) / (df['y'].max() - df['y'].min())
                    # transform to numpy array
                #     data = df.loc[:, ['frame','x','y']].values
                #     labels = df['cid'].to_numpy()


                    frame_positions = []

                    for frame in df['frame'].unique():
                        frame_data = df[df['frame'] == frame]

                        n_id = frame_data['id'].nunique()

                        frame_data["id_cumcounts"] = frame_data.groupby("id").cumcount()

                        frame_data.sort_values(["id_cumcounts", "id"], inplace=True)

                        frame_positions.append(frame_data[['frame', 'x', 'y']].to_numpy()[:n_id])

                # # transform to numpy array
                    try:

                        data = np.array(frame_positions)
                        t,n, m_plus = data.shape
                        st_data = data.reshape(t*n, m_plus)

                        true_labels = frame_data['cid'].to_list()[:n_id]
                        full_labels = np.tile(true_labels, t)

                        k = len(np.unique(true_labels))
                        
#                         if subset_option is True:
#                             t_full = t
#                             t = range(0,t_full,10)
                            
#                             data = data[t,:,:]
                            

                    except:
                        continue
                        
                    
                    
                    test = Test()
                    eps2 = alpha*t
                                        
#                     # grid search
#                     param_dict_dbscan = {'eps1': [0.02, 0.03, 0.04, 0.05],'eps2':[5, 25, 50, 100], 'min_samples': [2]}
#                     param_dict_agglo = {'eps2':[5, 25, 50, 100], 'n_clusters': [k]}
#                     param_dict_kmeans = {'eps2':[5, 25, 50, 100], 'n_clusters': [k]}
#                     # param_dict_optics = {'eps2':[5, 15, 25], 'min_cluster_size': [2], 'min_samples' : [2]}
#                     # param_dict_spectral = {'eps2':[5, 15, 25], 'n_clusters': [n_cluster]}
#                     # param_dict_affinity = {'eps2':[5, 15, 25]}
#                     param_dict_birch = {'eps2':[5, 25, 50, 100], 'threshold': [0.3, 0.5], 'n_clusters': [k]}
#         #             param_dict_hdbscan = {'eps2':[5, 25, 50, 100], 'min_cluster_size': [k], 'min_samples': [2]}
#                     ## ---

    
                    if method == 'dbscan':
                        print('running dbscan')
                        try:
#                             opt_param_dbscan = st_grid_search(estimator=ST_DBSCAN, split=True, X=data, param_dict=param_dict_dbscan, metric=metric, n_clusters = k, y=true_labels, frame_size=FRAME_SIZE, frame_overlap=FRAME_OVERLAP)
                            opt_param_dbscan = {'eps1':eps1, 'eps2': eps2, 'min_samples': 2}
                            dbscan = ST_DBSCAN(**opt_param_dbscan)
#                             if subset_option is True:
#                                 ami, full_ami, runtime = test.frame_split_cluster_fill_out(dbscan, data, FRAME_SIZE, FRAME_OVERLAP, t_full)
#                             else:
                            ami, full_ami, runtime = test.frame_split_cluster(dbscan, data, FRAME_SIZE, FRAME_OVERLAP)
                            print(ami, full_ami, runtime)
                            lt_ami.append(ami)
                            full_ami_list.append(full_ami)
                            runtime_list.append(runtime)
                            f.write(filename + ' ami: %f full_ami; %f runtime: %f \n\n' %(ami, full_ami, runtime))


                            print('running average', np.average(lt_ami), np.average(full_ami_list), np.average(runtime_list), '\n')

                #             logging.info('dataset: {}, method: {}, ami: {}, execution time: {}'.format(filename, dbscan, ami, runtime))
                        except TimeoutError:
                            print('timeout')
                #             logging.info('TimeoutError! dataset: {}, method: {}'.format(filename, 'DBSCAN'))
                            not_timed_out_dbscan = False
                            pass
                        except:
                            print('other error')
                #             logging.info('ComputationalError! dataset: {}, method: {}'.format(filename, 'DBSCAN'))
                            pass


                    ## ST_Agglomerative 
                    if method == 'agglomerative':
                        print('running agglomerative')
                        try:
#                             opt_param_agglo = st_grid_search(estimator=ST_Agglomerative, split=True, X=data, param_dict=param_dict_agglo, metric=metric, n_clusters = k, y=true_labels, frame_size=FRAME_SIZE, frame_overlap=FRAME_OVERLAP)
                            opt_param_agglo = {'eps1': eps1, 'eps2': eps2, 'n_clusters': k}
                            print(opt_param_agglo)
                            agglo = ST_Agglomerative(**opt_param_agglo)
                            ami, full_ami, runtime = test.frame_split_cluster(agglo, data, FRAME_SIZE, FRAME_OVERLAP)
                            lt_ami.append(ami)
                            full_ami_list.append(full_ami)
                            runtime_list.append(runtime)
                            f.write(filename + ' ami: %f full_ami: %f runtime: %f \n\n' %(ami, full_ami, runtime))

                            print('running average', np.average(lt_ami), np.average(full_ami_list), np.average(runtime_list), '\n')
                #             logging.info('dataset: {}, method: {}, ami: {}, execution time: {}'.format(filename, agglo, ami, runtime))
                        except TimeoutError:
                            print('timeout')
                #             logging.info('TimeoutError! dataset: {}, method: {}'.format(filename, 'Agglomerative'))
                            not_timed_out_agglo = False
                            pass
#                         except:
#                             print('other error')
#                 #             logging.info('ComputationalError! dataset: {}, method: {}'.format(filename, 'Agglomerative'))
#                             pass

                    ## K-MEANS
                    if method == 'kmeans':
                        print('running kmeans')
                        try:
#                             opt_param_kmeans = st_grid_search(estimator=ST_KMeans, split=True, X=data, param_dict=param_dict_kmeans, metric=metric, n_clusters = k, y=true_labels, frame_size=FRAME_SIZE, frame_overlap=FRAME_OVERLAP)
                            opt_param_kmeans = {'eps1': eps1, 'eps2': eps2, 'n_clusters': k}
                            kmeans = ST_KMeans(**opt_param_kmeans)
                            ami, full_ami, runtime = test.frame_split_cluster(kmeans, data, FRAME_SIZE, FRAME_OVERLAP)
                            lt_ami.append(ami)
                            full_ami_list.append(full_ami)
                            runtime_list.append(runtime)
                            f.write(filename + ' ami: %f full_ami: %f runtime: %f \n\n' %(ami, full_ami, runtime))

                            print('running average', np.average(lt_ami), np.average(full_ami_list), np.average(runtime_list), '\n')

            #logging.info('dataset: {}, method: {}, ami: {}, execution time: {}'.format(filename, kmeans, ami, runtime))
                        except TimeoutError:
                            print('timeout')
                            logging.info('TimeoutError! dataset: {}, method: {}'.format(filename, 'KMeans'))
                            not_timed_out_kmeans = False
                            pass
#                         except:
#                             print('other error')
#             #                 logging.info('ComputationalError! dataset: {}, method: {}'.format(filename, 'KMeans'))
#                             pass


                    ## BIRCH
                    if method == 'birch':
                        try:
#                             opt_param_birch = st_grid_search(estimator=ST_BIRCH, split=True, X=data, param_dict=param_dict_birch, metric=metric, n_clusters = k, y=true_labels, frame_size=FRAME_SIZE, frame_overlap=FRAME_OVERLAP)
                            opt_param_birch = {'eps1':eps1, 'eps2': eps2, 'n_clusters': k}
                            birch = ST_BIRCH(**opt_param_birch)
                            ami, full_ami, runtime = test.frame_split_cluster(birch, data, FRAME_SIZE, FRAME_OVERLAP)

                            lt_ami.append(ami)
                            full_ami_list.append(full_ami)
                            runtime_list.append(runtime)
                            f.write(filename + ' ami: %f full_ami: %f runtime: %f \n\n' %(ami, full_ami, runtime))

                            print('running average', np.average(lt_ami), np.average(full_ami_list), np.average(runtime_list), '\n')

                        except TimeoutError:
            #                 logging.info('TimeoutError! dataset: {}, method: {}'.format(filename, 'BIRCH'))
                            not_timed_out_birch = False
                            pass
                        except:
            #                 logging.info('ComputationalError! dataset: {}, method: {}'.format(filename, 'BIRCH'))
                            pass


                    ## HDBSCAN 
                    if method =='hdbscan':
                        try:
#                             opt_param_hdbscan = st_grid_search(estimator=ST_HDBSCAN, split=True, X=data, param_dict=param_dict_birch, metric=metric, n_clusters=k, y=true_labels, frame_size=FRAME_SIZE, frame_overlap=FRAME_OVERLAP)
                            opt_param_hdbscan = {'eps2': eps2, 'eps1': eps1, 'min_cluster_size': k}
#                             opt_param_hdbscan = {'eps2': 5, 'eps1': 0.5, 'n_clusters': 2}
                            hdbscan = ST_HDBSCAN(**opt_param_hdbscan)
                            ami, full_ami, runtime = test.frame_split_cluster(hdbscan, data, FRAME_SIZE, FRAME_OVERLAP)

                            lt_ami.append(ami)
                            full_ami_list.append(full_ami)
                            runtime_list.append(runtime)
                            f.write(filename + ' ami: %f full_ami: %f runtime: %f \n\n' %(ami, full_ami, runtime))

                            print('running average', np.average(lt_ami), np.average(full_ami_list), np.average(runtime_list), '\n')

                        except TimeoutError:
                            not_timed_out_hdbscan = False
                            pass 
                        except:
                            print('other error')
                            pass
            
        ami_average = np.average(lt_ami)
        full_ami_average = np.average(full_ami_list)
        runtime_average = np.average(runtime_list)
        f.write('ami average: %f full ami average: %f runtime average: %f' %(ami_average, full_ami_average, runtime_average))
        f.close()


            ## OPTICS 
            # if not_timed_out_optics:
            #     try:
            #         opt_param_optics = st_grid_search(estimator=ST_OPTICS, split=True, X=data, param_dict=param_dict_optics, metric='ami', y=labels, frame_size=FRAME_SIZE, frame_overlap=FRAME_OVERLAP)
            #         optics = ST_OPTICS(**opt_param_optics)
            #         ami, runtime = t.frame_split_cluster(optics, data, FRAME_SIZE, FRAME_OVERLAP)
            #         logging.info('dataset: {}, method: {}, ami: {}, execution time: {}'.format(filename, optics, ami, runtime))
            #     except TimeoutError:
            #         logging.info('TimeoutError! dataset: {}, method: {}'.format(filename, 'OPTICS'))
            #         not_timed_out_optics = False
            #         pass
            #     except:
            #         logging.info('ComputationalError! dataset: {}, method: {}'.format(filename, 'OPTICS'))
            #         pass

            ## SPECTRAL 
            # if not_timed_out_spectral:
            #     try:
            #         opt_param_spectral = st_grid_search(estimator=ST_SpectralClustering, split=True, X=data, param_dict=param_dict_spectral, metric='ami', y=labels, frame_size=FRAME_SIZE, frame_overlap=FRAME_OVERLAP)
            #         spectral = ST_SpectralClustering(**opt_param_spectral)
            #         ami, runtime = t.frame_split_cluster(spectral, data, FRAME_SIZE,FRAME_OVERLAP)
            #         logging.info('dataset: {}, method: {}, ami: {}, execution time: {}'.format(filename, spectral, ami, runtime))
            #     except TimeoutError:
            #         logging.info('TimeoutError! dataset: {}, method: {}'.format(filename, 'Spectral'))
            #         not_timed_out_spectral = False
            #         pass
            #     except:
            #         logging.info('ComputationalError! dataset: {}, method: {}'.format(filename, 'Spectral'))
            #         pass

            ## AFFINITY 
            # if not_timed_out_affinity:
            #     try:
            #         opt_param_affinity = st_grid_search(estimator=ST_AffinityPropagation, split=True, X=data, param_dict=param_dict_affinity, metric='ami', y=labels, frame_size=FRAME_SIZE, frame_overlap=FRAME_OVERLAP)
            #         affinity = ST_AffinityPropagation(**opt_param_affinity)
            #         ami, runtime = t.frame_split_cluster(affinity, data, FRAME_SIZE, FRAME_OVERLAP)
            #         logging.info('dataset: {}, method: {}, ami: {}, execution time: {}'.format(filename, affinity,  ami, runtime))
            #     except TimeoutError:
            #         logging.info('TimeoutError! dataset: {}, method: {}'.format(filename, 'Affinity'))
            #         not_timed_out_affinity = False
            #         pass
            #     except:
            #         logging.info('ComputationalError! dataset: {}, method: {}'.format(filename, 'Affinity'))
            #         pass

/var/folders/xz/_nh5vnzd6w35js_89b99q6r40000gn/T/ipykernel_8279/2556300516.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frame_data["id_cumcounts"] = frame_data.groupby("id").cumcount()
/opt/anaconda3/envs/tkm/lib/python3.9/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


running dbscan
Could not find k clusters
-1.3624583240492536e-15 6.255954471249542e-15 0.12880611419677734
running average -1.3624583240492536e-15 6.255954471249542e-15 0.12880611419677734 



/var/folders/xz/_nh5vnzd6w35js_89b99q6r40000gn/T/ipykernel_8279/2556300516.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frame_data["id_cumcounts"] = frame_data.groupby("id").cumcount()
/opt/anaconda3/envs/tkm/lib/python3.9/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


running dbscan
Could not find k clusters
1.7156587852386127e-15 8.951263227049689e-17 0.15440702438354492
running average 1.7660023059467955e-16 3.1727335517600192e-15 0.14160656929016113 



/var/folders/xz/_nh5vnzd6w35js_89b99q6r40000gn/T/ipykernel_8279/2556300516.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frame_data["id_cumcounts"] = frame_data.groupby("id").cumcount()
/opt/anaconda3/envs/tkm/lib/python3.9/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


...

Could not find k clusters
running average 0.545008308886023 0.3173009903534851 7.6537794230673555 



/var/folders/xz/_nh5vnzd6w35js_89b99q6r40000gn/T/ipykernel_8279/2556300516.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frame_data["id_cumcounts"] = frame_data.groupby("id").cumcount()
/opt/anaconda3/envs/tkm/lib/python3.9/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


running average 0.544884382675477 0.3174323930048764 7.62831718966646 



/var/folders/xz/_nh5vnzd6w35js_89b99q6r40000gn/T/ipykernel_8279/2556300516.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frame_data["id_cumcounts"] = frame_data.groupby("id").cumcount()
/opt/anaconda3/envs/tkm/lib/python3.9/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


Could not find k clusters
running average 0.5438139533010911 0.31595752417600903 7.622862292007661 



In [38]:
frame_size = 100
frame_overlap= 10
eps1 = 0.05
alpha = 0.60
metric = 'static'

df = pd.read_csv('../../full_data/reynolds_600_44.csv')
df['x'] = (df['x'] - df['x'].min()) / (df['x'].max() - df['x'].min())
df['y'] = (df['y'] - df['y'].min()) / (df['y'].max() - df['y'].min())
frame_positions = []

for frame in df['frame'].unique():
    frame_data = df[df['frame'] == frame]

    n_id = frame_data['id'].nunique()

    frame_data["id_cumcounts"] = frame_data.groupby("id").cumcount()

    frame_data.sort_values(["id_cumcounts", "id"], inplace=True)

    frame_positions.append(frame_data[['frame', 'x', 'y']].to_numpy()[:n_id])

# # transform to numpy array


data = np.array(frame_positions)
t,n, m_plus = data.shape
st_data = data.reshape(t*n, m_plus)

true_labels = frame_data['cid'].to_list()[:n_id]
full_labels = np.tile(true_labels, t)

k = len(np.unique(true_labels))


eps2 = alpha*t

print('running agglomerative')

opt_param_agglo = {'eps1': eps1, 'eps2': eps2, 'n_clusters':k}
dbscan = ST_Agglomerative(**opt_param_agglo)
# dbscan.st_fit_frame_split(st_data, frame_size = frame_size, frame_overlap=frame_overlap)
dbscan.st_fit(st_data)
weights = dbscan.labels.reshape((t,n)).T

/var/folders/xz/_nh5vnzd6w35js_89b99q6r40000gn/T/ipykernel_54161/1427254470.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frame_data["id_cumcounts"] = frame_data.groupby("id").cumcount()
/opt/anaconda3/envs/tkm/lib/python3.9/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


running agglomerative


In [41]:
t,n,k, weights.shape

(600, 37, 3, (37, 600))

In [40]:
weights[5,:]

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [25]:
from sklearn.metrics import f1_score
from TKM_long_term_clusters import find_k_clusters, similarity_matrix

# f1 = f1_score(np.tile(true_labels, t), (weights.T).reshape(t * n), average=None)
tot_ami = adjusted_mutual_info_score(np.tile(true_labels, t), (weights.T).reshape(t * n))
print(tot_ami)
print((weights.T).reshape(t * n))
print(np.unique(weights), np.unique(true_labels))
print(np.tile(true_labels, t))



criteria_mat = similarity_matrix(weights)

ltc = find_k_clusters(k=k, criteria_mat=criteria_mat, threshold_change=.05, verbose=True)

pred_labels = np.zeros(n)

for i, cluster in enumerate(ltc):
    pred_labels[cluster] = i

ami = adjusted_mutual_info_score(true_labels, pred_labels)

print(ami)

0.045034155937008165
[  1   1   1 ... -99 -99 -99]
[-99   0   1   2] [0 1 2]
[0 0 0 ... 2 2 2]
Threshold for  3  clusters:  0.9000000000000001
1.0


In [18]:
for i in range(t):
    print((weights.T).reshape(t * n)[i*n:(i+1)*n])

[1 1 1 2 1 1 1 1 1 1 1 1 1 2 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[1 1 1 2 1 2 1 1 1 1 1 1 1 2 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[1 1 1 2 2 2 1 2 1 1 1 1 1 2 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[1 1 1 2 2 2 1 2 1 2 1 1 1 2 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[1 1 1 2 2 2 1 2 1 2 1 2 1 2 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[2 2 1 2 2 2 1 2 1 2 1 2 1 2 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[2 2 2 2 2 2 1 2 2 2 1 2 1 2 2 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1]
[2 2 2 2 2 2 2 2 2 2 1 2 2 2 2 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 0 1]
[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 0 0 0]
[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 0 0 0 0]
[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 1 1 1 0 1 1 1 1 1 1 1 0 0 0 1 1 0 0 0 0]
[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 1 1 1 0 0 0 1 1 1 1 1 0 0 0 1 0 0 0 0 0]
[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 1 1 1 0 0 0 1 1 1 1 1 0 0 0 0 0 0 0 0 0]
[2 2 2 2 2 2